In [1]:
from sklearn import datasets
from sklearn.model_selection import KFold
import numpy as np

from itertools import combinations

In [2]:
DATA_PER_CLASS = 50

# Base classifier model

In [3]:
class BaseClassifier:
    def __init__(self, data: np.ndarray, target: np.ndarray):
        self.data = data
        self.target = target

        self._w = {}

    def __str__(self):
        pass

    # def getW(self):
    #     return self._w

    # def k_fold_split(self, n_folds: int) -> list:
    #     pass

    def fit(self) -> list:
        pass

    def predict(self, test: list) -> float:
        pass

    def test(self, k) -> list:
        pass

# 2-class discriminant function classifier using one-vs-one combination

In [4]:
# class_pairs = list(combinations(np.unique(target), 2))
# class_pairs

In [5]:
class TwoClassesOneOneClassifier(BaseClassifier):
    def __init__(self, data, target):
        super().__init__(data,target)
    
    def __str__(self):
        return f"TwoClassesOneOneClassifier(numClasses = {len(np.unique(self.target))})"
    
    def fit(self, train_data, train_target):
        class_pairs = list(combinations(np.unique(train_target), 2))
        
        for (first, second) in class_pairs:
            idx = np.where((train_target == first) | (train_target == second))
            tempX = train_data[idx]
            tempT = train_target[idx]
            tempT[tempT == first] = -1
            tempT[tempT == second] = 1
            self._w[(first, second)] = np.dot(np.linalg.pinv(tempX),tempT)
        return self._w

    def predict(self, test, pair_id):
        # print(pair_id)
        # print(np.dot(np.transpose(self._w[pair_id]),np.array(test)))
        return int(np.dot(np.transpose(self._w[pair_id]),np.array(test)) > 0)
    
    def test(self, k):
        kf = KFold(n_splits=k, shuffle=True, random_state=50)
        acc_by_folds = []

        for train_index, test_index in kf.split(self.data):
            train_data, test_data = self.data[train_index], self.data[test_index]
            train_target, test_target = self.target[train_index], self.target[test_index]
            
            class_pairs = list(combinations(np.unique(test_target), 2))
            self.fit(train_data,train_target)
            acc_by_pairs = []
    
            for (first, second) in class_pairs:
                idx = np.where((test_target == first) | (test_target == second))
                tempX = test_data[idx]
                tempT = test_target[idx]
                tempT[tempT == first] = 0
                tempT[tempT == second] = 1
                acc = 0
                # print(tempT)
                for i, test in enumerate(tempX):
                    if self.predict(test, (first, second)) == tempT[i]:
                        acc += 1
                acc_by_pairs.append(acc/len(tempX))
            acc_by_folds.append(acc_by_pairs)

        return acc_by_folds
                
        

# 2-class discriminant function classifier using one-vs-rest combination

In [6]:
class TwoClassesOneRestClassifier(BaseClassifier):
    def __init__(self, data, target):
        super().__init__(data,target)
    
    def __str__(self):
        return f"TwoClassesOneRestClassifier(numClasses = {len(np.unique(self.target))})"
    
    def fit(self, train_data, train_target):
        for first in range(3):
            idx = np.where((train_target == first) | (train_target != first))
            tempX = train_data[idx]
            tempT = train_target[idx]
            tempT[tempT == first] = -1
            tempT[tempT != first] = 1
            self._w[first] = np.dot(np.linalg.pinv(tempX),tempT)
        return self._w

    def predict(self, test, pair_id):
        return int(np.dot(np.transpose(self._w[pair_id]),np.array(test)) > 0)
    
    def test(self, k):
        kf = KFold(n_splits=k, shuffle=True, random_state=50)
        acc_by_folds = []

        for train_index, test_index in kf.split(self.data):
            train_data, test_data = self.data[train_index], self.data[test_index]
            train_target, test_target = self.target[train_index], self.target[test_index]
            
            # class_pairs = list(combinations(np.unique(test_target), 2))
            self.fit(train_data,train_target)
            acc_by_pairs = []
    
            for first in range(3):
                idx = np.where((test_target == first) | (test_target != first))
                tempX = test_data[idx]
                tempT = test_target[idx]
                tempT[tempT == first] = 0
                tempT[tempT != first] = 1
                acc = 0
                # print(tempT)
                for i, test in enumerate(tempX):
                    # print("test:",test)
                    # t = self.predict(test, (first, second))
                    # print(t)
                    if self.predict(test, first) == tempT[i]:
                        acc += 1
                acc_by_pairs.append(acc/len(tempX))
            acc_by_folds.append(acc_by_pairs)

        return acc_by_folds

# 3-class discriminant function classifier

In [7]:
class ThreeClassesClassifier(BaseClassifier):
    pass

# 3-class Fisher's discriminant function classifier

In [8]:
class ThreeClassesFisherClassifier(BaseClassifier):
    pass

# 3-class Bayesian discriminant function classifier

In [9]:
class ThressClassesBayesianClassifier(BaseClassifier):
    pass

# Load data and predict

In [10]:
# def test(classifier: BaseClassifier, input: list) -> int:
#     pass

In [11]:
def k_fold_test(classifier: BaseClassifier) -> list:
    pass

In [12]:
iris = datasets.load_iris()
data = iris.data
target = iris.target

data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [13]:
base = TwoClassesOneRestClassifier(data,target)
# input = [6.5, 3. , 5.2, 2. ]
print(base.test(5))

[[0.7, 1.0, 1.0], [0.6, 1.0, 1.0], [0.6666666666666666, 1.0, 1.0], [0.7333333333333333, 1.0, 1.0], [0.6333333333333333, 1.0, 1.0]]


In [14]:
# pred_result = test(base,input)
# pred_result

In [15]:
test_result = k_fold_test(base)
test_result

In [16]:
# matX = np.array([
#     [1, 0, 3, 4, 9],
#     [2, 2, 3, 6, 5],
#     [-1, -2, 3, -4, 7],
#     [1, 2, -3, 4, 6],
# ])
# matT = np.array([0,1,1,0])
# xTx = np.dot(np.transpose(matX),matX)
# w = np.dot(np.dot(np.linalg.inv(xTx),np.transpose(matX)),matT)
# print(w)
# print(np.linalg.det(xTx))

In [17]:
# print(int(True))